<a href="https://colab.research.google.com/github/EANimesha/Coursera_Capstone/blob/main/Coursera_Capstone_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Finding a suitable place to open a shopping mall in Delhi.**

In [9]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium
import requests

1. Loading Datasets

In [2]:
#loading the neighborhood data set. I get the data set from kaggle
delhi = pd.read_csv('/content/sample_data/delhi_dataSet.csv',skipinitialspace=True, usecols=['Borough','Neighborhood','latitude','longitude'])

In [3]:
#Drop rows containing NaN values
delhi = delhi.dropna()

In [4]:
delhi.head()

,Borough,Neighborhood,latitude,longitude
0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,North West Delhi,Azadpur,28.707657,77.175547
3,North West Delhi,Bawana,28.799660,77.032885
5,North West Delhi,Dhaka,39.031714,-90.261223


2. Visualizing neighborhood on a map


In [5]:
#get cordinates of Delhi India
address = 'Delhi, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Colombo are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Colombo are 28.6517178, 77.2219388.


In [6]:
# create map of Delhi using latitude and longitude values
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood, borough in zip(delhi['latitude'], delhi['longitude'], delhi['Neighborhood'], delhi['Borough']):
    label = '{},{}'.format(borough,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_delhi)  
    
map_delhi

3. Using Forsquare API to explore venues around Delhi

In [ ]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [10]:
# get the top 100 venues that are within a radius of 2000 meters.
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(delhi['latitude'], delhi['longitude'], delhi['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,

            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [11]:
venues_df = pd.DataFrame(venues)

venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6732, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adarsh Nagar,28.614192,77.071541,Shree Maakhan,28.625825,77.068608,Indian Restaurant
1,Adarsh Nagar,28.614192,77.071541,Cafe Coffee Day,28.622388,77.088353,Café
2,Adarsh Nagar,28.614192,77.071541,Domino's Pizza,28.630000,77.080000,Pizza Place
3,Adarsh Nagar,28.614192,77.071541,Cafe Coffee Day,28.629794,77.079771,Coffee Shop
4,Adarsh Nagar,28.614192,77.071541,Subway,28.630026,77.079541,Sandwich Place


In [12]:
# check how many venues were returned for each neighorhood
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adarsh Nagar,12,12,12,12,12,12
Alaknanda,57,57,57,57,57,57
Anand Vihar,36,36,36,36,36,36
Ashok Nagar,2,2,2,2,2,2
Ashok Vihar,31,31,31,31,31,31
Azadpur,27,27,27,27,27,27
Babarpur,5,5,5,5,5,5
Badarpur,5,5,5,5,5,5
Bali Nagar,43,43,43,43,43,43


In [13]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 217 uniques categories.


In [14]:
venues_df['VenueCategory'].unique()[:50]

array(['Indian Restaurant', 'Café', 'Pizza Place', 'Coffee Shop',
       'Sandwich Place', 'Gym', 'Pool', 'Bakery', 'Fast Food Restaurant',
       'Gym / Fitness Center', 'Train Station', 'Asian Restaurant',
       'Snack Place', 'Chinese Restaurant', 'Department Store',
       'Donut Shop', 'South Indian Restaurant', 'Food Truck',
       'Convenience Store', 'Bus Station', 'Lake', 'Dessert Shop',
       'Market', 'Restaurant', 'Clothing Store',
       'North Indian Restaurant', 'Multiplex', 'Light Rail Station',
       'Shopping Mall', 'Playground', 'Business Service', 'Dive Bar',
       'Hotel', 'ATM', 'Athletics & Sports', 'Diner',
       'Arts & Crafts Store', 'Smoke Shop', 'Music Venue',
       'History Museum', 'Spa', 'Garden', 'Karnataka Restaurant',
       'BBQ Joint', 'Northeast Indian Restaurant', 'Hotel Bar', 'Lounge',
       'Sculpture Garden', 'Planetarium', 'Moroccan Restaurant'],
      dtype=object)

In [15]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

4. Analyze Venues

In [16]:
# one hot encoding
delhi_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
delhi_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [delhi_onehot.columns[-1]] + list(delhi_onehot.columns[:-1])
delhi_onehot = delhi_onehot[fixed_columns]

print(delhi_onehot.shape)
delhi_onehot.head()

(6732, 218)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Burmese Restaurant,Bus Station,Business Center,Business Service,Cafeteria,Café,Campground,Castle,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hindu Temple,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Luggage Store,Market,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Recreation Center,Rental Car Location,Restaurant,River,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Women's Store,Yoga Studio,Zoo
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [18]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

delhi_grouped = delhi_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(delhi_grouped.shape)
delhi_grouped

(154, 218)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Burmese Restaurant,Bus Station,Business Center,Business Service,Cafeteria,Café,Campground,Castle,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hindu Temple,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Luggage Store,Market,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Recreation Center,Rental Car Location,Restaurant,River,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Women's Store,Yoga Studio,Zoo
0,Adarsh Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0

In [21]:
len(delhi_grouped[delhi_grouped["Shopping Mall"] > 0])
# There are 48 shopping malls

48

In [23]:
#Create a new DataFrame for Shopping Mall data only
delhi_malls = delhi_grouped[["Neighborhoods","Shopping Mall"]]

In [24]:
delhi_malls.head()

,Neighborhoods,Shopping Mall
0,Adarsh Nagar,0.000000
1,Alaknanda,0.000000
2,Anand Vihar,0.055556
3,Ashok Nagar,0.000000
4,Ashok Vihar,0.000000


5. Cluster Neighborhoods with K-means Clustering

In [25]:
# set number of clusters
kclusters = 3

delhi_clustering = delhi_malls.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(delhi_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 1, 1, 0, 1, 1, 1, 1], dtype=int32)

In [26]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
delhi_merged = delhi_malls.copy()

# add clustering labels
delhi_merged["Cluster Labels"] = kmeans.labels_

In [27]:
delhi_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
delhi_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Adarsh Nagar,0.000000,1
1,Alaknanda,0.000000,1
2,Anand Vihar,0.055556,0
3,Ashok Nagar,0.000000,1
4,Ashok Vihar,0.000000,1


In [28]:
# merge delhi_merged with delhi data to add latitude/longitude for each neighborhood
delhi_merged = delhi_merged.join(delhi.set_index('Neighborhood'), on='Neighborhood')

print(delhi_merged.shape, delhi.shape)
delhi_merged.head() 

(163, 6) (163, 4)


,Neighborhood,Shopping Mall,Cluster Labels,Borough,latitude,longitude
0,Adarsh Nagar,0.000000,1,North West Delhi,28.614192,77.071541
1,Alaknanda,0.000000,1,South Delhi,28.529336,77.251632
2,Anand Vihar,0.055556,0,East Delhi,28.625691,77.101941
3,Ashok Nagar,0.000000,1,West Delhi,39.031714,-90.261223
4,Ashok Vihar,0.000000,1,North West Delhi,28.699453,77.184826


In [29]:
# sort data by cluster labels
print(delhi_merged.shape)
delhi_merged.sort_values(["Cluster Labels"], inplace=True)
delhi_merged

(163, 6)


,Neighborhood,Shopping Mall,Cluster Labels,Borough,latitude,longitude
78,Malviya Nagar,0.040000,0,South Delhi,28.533920,77.212447
30,Fateh Nagar,0.047619,0,West Delhi,28.631615,77.100878
124,Sant Nagar,0.064516,0,North Delhi,28.646418,77.091188
34,Geeta Colony,0.058824,0,East Delhi,28.650101,77.275921
122,Saket,0.040000,0,South Delhi,28.524411,77.213725
114,Rajouri Garden,0.050847,0,West Delhi,28.642152,77.116060
113,Rajokri,0.036364,0,South West Delhi,28.513170,77.110766
44,INA Colony,0.029851,0,New Delhi,28.579472,77.212451
112,Punjabi Bagh,0.052632,0,West Delhi,28.668945,77.132461
110,Preet Vihar,0.066667,0,East Delhi,28.637753,77.295175


In [30]:
# create map to visualize cluster data
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(delhi_merged['latitude'], delhi_merged['longitude'], delhi_merged['Neighborhood'], delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

6. Examine Clusters

In [34]:
# Cluster 0
print (delhi_merged.loc[delhi_merged['Cluster Labels'] == 0].shape)
delhi_merged.loc[delhi_merged['Cluster Labels'] == 0]

(32, 6)


,Neighborhood,Shopping Mall,Cluster Labels,Borough,latitude,longitude
78,Malviya Nagar,0.040000,0,South Delhi,28.533920,77.212447
30,Fateh Nagar,0.047619,0,West Delhi,28.631615,77.100878
124,Sant Nagar,0.064516,0,North Delhi,28.646418,77.091188
34,Geeta Colony,0.058824,0,East Delhi,28.650101,77.275921
122,Saket,0.040000,0,South Delhi,28.524411,77.213725
114,Rajouri Garden,0.050847,0,West Delhi,28.642152,77.116060
113,Rajokri,0.036364,0,South West Delhi,28.513170,77.110766
44,INA Colony,0.029851,0,New Delhi,28.579472,77.212451
112,Punjabi Bagh,0.052632,0,West Delhi,28.668945,77.132461
110,Preet Vihar,0.066667,0,East Delhi,28.637753,77.295175


In [36]:
#Cluster 1
print (delhi_merged.loc[delhi_merged['Cluster Labels'] == 1].shape)
delhi_merged.loc[delhi_merged['Cluster Labels'] == 1]

(125, 6)


,Neighborhood,Shopping Mall,Cluster Labels,Borough,latitude,longitude
88,Najafgarh,0.000000,1,South West Delhi,28.612304,76.982391
104,Paschim Vihar,0.000000,1,West Delhi,28.669578,77.095956
147,Vasundhara Enclave,0.000000,1,East Delhi,28.601726,77.321122
102,Palam,0.000000,1,South West Delhi,28.567173,77.134032
85,Moti Bagh,0.000000,1,South West Delhi,28.577879,77.174180
101,Paharganj,0.000000,1,Central Delhi,28.641926,77.221750
101,Paharganj,0.000000,1,North Delhi,28.641926,77.221750
100,Okhla,0.000000,1,South Delhi,28.563662,77.289055
99,Nizamuddin West,0.000000,1,South Delhi,28.588365,77.244955
148,Vikas Nagar,0.000000,1,West Delhi,28.644009,77.054470


In [35]:
#Cluster 2
print(delhi_merged.loc[delhi_merged['Cluster Labels'] == 2].shape)
delhi_merged.loc[delhi_merged['Cluster Labels'] == 2]

(6, 6)


,Neighborhood,Shopping Mall,Cluster Labels,Borough,latitude,longitude
66,Kirby Place,0.111111,2,South West Delhi,28.611741,77.128527
74,Laxmi Nagar,0.086957,2,East Delhi,28.630309,77.277563
24,Delhi Cantonment,0.111111,2,South West Delhi,28.583536,77.125985
132,Shakarpur,0.095238,2,East Delhi,28.629489,77.281061
118,Rithala,0.083333,2,North West Delhi,28.720806,77.107181
83,Mehrauli,0.076923,2,South Delhi,28.521826,77.178323


# **Conclusion**

Most of the shopping malls are concentrated around the new delhi. Highest number of shopping malls in Cluster 1, moderate amount in cluster 0 and lowest amount in cluster 2.   This represents a great opportunity and high potential areas to open new shopping malls, as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 1 are likely suffering from intense competition because of oversupply and a high concentration of shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighbourhoods in cluster 0 with little to no competition.